In [24]:
M = 20

f1 = [0.00, 0.09, 0.09, 0.09, 0.09, 0.09, 0.09, 0.09, 0.09, 0.09, 0.09, 0.10]
f2 = [0.00, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10, 0.45]
g1 = [0.00, 0.01, 0.04, 0.07, 0.10, 0.13, 0.30, 0.13, 0.10, 0.07, 0.04, 0.01]
g2 = [0.00, 0.20, 0.60, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]

In [8]:
def expected_rev_city_1(available_cars):
    expected_rev = 0

    for d_same in range(1, 12):
        for d_other in range(1, 12):
            prob = f1[d_same] * g1[d_other]

            if prob == 0:
                continue
                
            actual_rent_same = min(d_same, available_cars)

            cars_leftover = available_cars - actual_rent_same

            actual_rent_other = min(d_other, cars_leftover)

            money = (actual_rent_same * 50) + (actual_rent_other * 45)

            expected_rev += prob * money
    
    return expected_rev

In [11]:
def expected_rev_city_2(available_cars):
    expected_rev = 0

    for d_same in range(1, 12):
        for d_other in range(1, 12):
            prob = f2[d_same] * g2[d_other]

            if prob == 0:
                continue
                
            actual_rent_same = min(d_same, available_cars)

            cars_leftover = available_cars - actual_rent_same

            actual_rent_other = min(d_other, cars_leftover)

            money = (actual_rent_same * 50) + (actual_rent_other * 45)

            expected_rev += prob * money
    
    return expected_rev

In [ ]:
V = [0.0] * 21
gamma = 0.9
policy = [0] * 21

In [25]:
def calculate_action_value(state, action, V, gamma):
    cost = -(abs(action) * 60)
    
    avail_1 = state - max(0, action)
    avail_2 = (20 - state) - max(0, -action)

    total_expected_return = cost

    for d1_same in range(1, 12):
        for d1_other in range(1, 12):
            for d2_same in range(1, 12):
                for d2_other in range(1, 12):

                    prob = f1[d1_same] * g1[d1_other] * f2[d2_same] * g2[d2_other]

                    if prob == 0:
                        continue

                    real_rent_1_same = min(d1_same, avail_1)
                    avail_for_oneway_1 = avail_1 - real_rent_1_same
                    real_rent_1_other = min(d1_other, avail_for_oneway_1)

                    real_rent_2_same = min(d2_same, avail_2)
                    avail_for_oneway_2 = avail_2 - real_rent_2_same
                    real_rent_2_other = min(d2_other, avail_for_oneway_2)

                    revenue = 50 * (real_rent_1_same + real_rent_2_same) + 45 * (real_rent_1_other + real_rent_2_other)

                    cars_staying_in_1 = avail_1 - real_rent_1_other
                    cars_coming_from_2_rentals = real_rent_2_other

                    cars_coming_from_2_moves = max(0, -action)
                    s_prime = cars_staying_in_1 + cars_coming_from_2_rentals + cars_coming_from_2_moves

                    if s_prime > 20:
                        s_prime = 20

                    if s_prime < 0:
                        s_prime = 0

                    total_expected_return += prob * (revenue + gamma * V[int(s_prime)])
    
    return total_expected_return

In [22]:
theta = 0.1
iteration = 0

In [23]:
print("Starting Value Iteration...")

while True:
    iteration += 1
    delta = 0
    
    for s in range(21):
        old_v = V[s]
        
        best_val = -float('inf')
        best_action = 0
        
        min_action = -(20 - s) 
        max_action = s
        
        for a in range(min_action, max_action + 1):
            
            val = calculate_action_value(s, a, V, gamma)
            
            if val > best_val:
                best_val = val
                best_action = a
        
        V[s] = best_val
        policy[s] = best_action
        
        diff = abs(old_v - V[s])
        if diff > delta:
            delta = diff
            
    print(f"Iteration {iteration}: Max value change = {delta:.4f}")
    
    if delta < theta:
        print("Convergence achieved!")
        break

print("\nOptimal Policy (Cars to move from City 1 -> City 2):")
for s in range(21):
    print(f"State {s} (Cars in City 1): Move {policy[s]}")

Starting Value Iteration...
Iteration 1: Max value change = 298.1783
Iteration 2: Max value change = 28.9236
Iteration 3: Max value change = 24.0407
Iteration 4: Max value change = 20.1590
Iteration 5: Max value change = 16.8431
Iteration 6: Max value change = 14.0666
Iteration 7: Max value change = 11.7481
Iteration 8: Max value change = 9.8119
Iteration 9: Max value change = 8.1948
Iteration 10: Max value change = 6.8442
Iteration 11: Max value change = 5.7162
Iteration 12: Max value change = 4.7741
Iteration 13: Max value change = 3.9873
Iteration 14: Max value change = 3.3301
Iteration 15: Max value change = 2.7813
Iteration 16: Max value change = 2.3229
Iteration 17: Max value change = 1.9401
Iteration 18: Max value change = 1.6203
Iteration 19: Max value change = 1.3533
Iteration 20: Max value change = 1.1302
Iteration 21: Max value change = 0.9440
Iteration 22: Max value change = 0.7884
Iteration 23: Max value change = 0.6585
Iteration 24: Max value change = 0.5499
Iteration 25: